In [ ]:
import MySQLdb
from pandas.io import sql
# from zmq import int_sockopt_names
conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor() 

In [ ]:
import pandas as pd
import numpy as np
# pd.set_option('display.max_rows)
df_obs = pd.read_sql('select * from `obs` where concept_id = 1482681', con = conn)
df_obs

In [ ]:
df_obs[['person_id','value_numeric']]

In [ ]:
df_encounter = pd.read_sql('select * from `encounter`', con = conn) 
df_encounter

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()
df_icustays_mimic = pd.read_sql('select * from icustays', con = mimicconn) 
df_icustays_mimic

In [ ]:
dict_ = {'Trauma SICU (TSICU)' : 2, 'Neuro Stepdown' : 3, 'Coronary Care Unit (CCU)' : 4, 
         'Neuro Intermediate' : 5, 'Medical Intensive Care Unit (MICU)':6, 'Cardiac Vascular Intensive Care Unit (CVICU)': 7,
        'Surgical Intensive Care Unit (SICU)':8, 'Neuro Surgical Intensive Care Unit (Neuro SICU)': 9, 
        'Medical/Surgical Intensive Care Unit (MICU/SICU)': 10}
df_icustays_mimic=df_icustays_mimic.replace({"first_careunit": dict_})
df_icustays_mimic=df_icustays_mimic.replace({"last_careunit": dict_})

In [ ]:
df_obs_los = df_icustays_mimic.rename({'stay_id':'encounter_id', 'los':'value_numeric', 'subject_id':'person_id', 'intime':'obs_datetime', 'first_careunit':'location_id' }, axis='columns')
df_obs_los

In [ ]:
import uuid
df_obs_los['date_created'] = '2022-01-01'
df_obs_los['uuid'] = [uuid.uuid4() for x in range(len(df_obs_los))]

In [ ]:
df_obs_los =  df_obs_los.drop(columns =['last_careunit', 'outtime', 'hadm_id' ])
df_obs_los['concept_id'] = 

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in df_obs_los.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_obs_los.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `obs` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

obs_los_concept_id = 'UPDATE obs SET concept_id= 1482681'
openmrscursor.execute(obs_los_concept_id)
obs_los_order_id = 'UPDATE obs SET order_id = NULL'
openmrscursor.execute(obs_los_concept_id)
openmrscursor.execute('UPDATE obs SET creator=1')
conn.commit() 

In [ ]:
# openmrscursor.execute('Delete from obs')
# conn.commit() 